In [1]:
from nba_api.stats.endpoints import leaguegamefinder  
import dataframe_image as dfi
import pandas as pd
import numpy as np
from threading import Thread, Event
import requests
import time
from bs4 import BeautifulSoup, Comment
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
#All NBA Games from 1983-current
from timeit import default_timer as timer
from datetime import timedelta
import time

start = timer()
columns = ['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS']
gamesByTeams = pd.DataFrame({}, columns=columns)
for n in range(1983, 2020):
    date = f"{n}-{str(n+1)[2:]}"
    print(date)
    try:
        gamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00', season_nullable=date, season_type_nullable="Regular Season")
    except:
        print('failed, trying again')
        time.sleep(60)
        try:
            gamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00', season_nullable=date, season_type_nullable="Regular Season")
        except:
            print('failed again, breaking')
            break       
    allGames = gamefinder.get_data_frames()[0]
    allGames = allGames.reindex(index=allGames.index[::-1]).reset_index(drop=True)
    gamesByTeams = pd.concat([gamesByTeams, allGames]) 
    time.sleep(12)    
gamesByTeams = gamesByTeams.reset_index(drop=True) 
gamesByTeams['WL'] = [1 if game['WL'] == 'W' else 0 for index, game in gamesByTeams.iterrows()]
gamesByTeams.to_csv('nba-teams-1984-2019-ordered.csv')

end = timer()
print(timedelta(seconds=end-start))

In [46]:
gamesByTeams

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,21983,1610612764,WAS,Washington Bullets,28300003,1983-10-28,WAS @ PHL,0,240,114,...,0.759,10.0,26.0,36,32,9.0,3,21,35,NaN
1,21983,1610612739,CLE,Cleveland Cavaliers,28300008,1983-10-28,CLE vs. NYK,0,240,106,...,0.650,18.0,17.0,35,22,9.0,8,18,25,NaN
2,21983,1610612749,MIL,Milwaukee Bucks,28300001,1983-10-28,MIL vs. IND,1,240,104,...,0.769,21.0,39.0,60,24,12.0,13,19,22,NaN
3,21983,1610612754,IND,Indiana Pacers,28300001,1983-10-28,IND @ MIL,0,240,83,...,0.619,24.0,29.0,53,27,7.0,4,19,22,NaN
4,21983,1610612747,LAL,Los Angeles Lakers,28300002,1983-10-28,LAL @ KCK,1,240,117,...,0.760,9.0,29.0,38,22,6.0,5,17,16,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84813,22022,1610612765,DET,Detroit Pistons,22200347,2022-12-04,DET vs. MEM,0,240,112,...,0.833,8.0,31.0,39,26,7.0,7,14,24,-10.0
84814,22022,1610612764,WAS,Washington Wizards,22200349,2022-12-04,WAS vs. LAL,0,240,119,...,0.733,11.0,33.0,44,24,10.0,8,10,23,-11.0
84815,22022,1610612752,NYK,New York Knicks,22200348,2022-12-04,NYK vs. CLE,1,239,92,...,0.810,14.0,40.0,54,17,6.0,3,20,16,11.0
84816,22022,1610612763,MEM,Memphis Grizzlies,22200347,2022-12-04,MEM @ DET,1,240,122,...,0.724,12.0,34.0,46,28,9.0,5,13,21,10.0


In [49]:
# Create HOME and AWAY Columns
columns = gamesByTeams.columns.to_series().add_prefix("HOME_").index.to_list() + gamesByTeams.columns.to_series().add_prefix("AWAY_").index.to_list()
transformed_games = pd.DataFrame({}, columns=columns)
game_ids = gamesByTeams['GAME_ID'].unique()
for id in game_ids:
        grouped_game = gamesByTeams.loc[gamesByTeams['GAME_ID'] == id]
        if 'vs' in grouped_game.iloc[0, 6]:
            home = grouped_game.iloc[[0]]
            home = home.add_prefix('HOME_')
            away = grouped_game.iloc[[1]]
            away = away.add_prefix('AWAY_')
        else:
            home = grouped_game.iloc[[1]]
            home = home.add_prefix('HOME_')
            away = grouped_game.iloc[[0]]
            away = away.add_prefix('AWAY_')
        game_played = pd.concat([home,away.set_index(home.index)], axis=1)
        transformed_games = pd.concat([transformed_games, game_played])
transformed_games['HOME_GAME_DATE'] = [datetime.strptime(game['HOME_GAME_DATE'], '%Y-%m-%d') for index, game in transformed_games.iterrows()]
transformed_games['AWAY_GAME_DATE'] = [datetime.strptime(game['AWAY_GAME_DATE'], '%Y-%m-%d') for index, game in transformed_games.iterrows()]
transformed_games = transformed_games.reset_index(drop=True) 

In [50]:
transformed_games

,HOME_SEASON_ID,HOME_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NAME,HOME_GAME_ID,HOME_GAME_DATE,HOME_MATCHUP,HOME_WL,HOME_MIN,HOME_PTS,...,AWAY_FT_PCT,AWAY_OREB,AWAY_DREB,AWAY_REB,AWAY_AST,AWAY_STL,AWAY_BLK,AWAY_TOV,AWAY_PF,AWAY_PLUS_MINUS
0,21983,1610612755,PHL,Philadelphia 76ers,28300003,1983-10-28,PHL vs. WAS,1,240,117,...,0.759,10.0,26.0,36,32,9.0,3,21,35,NaN
1,21983,1610612739,CLE,Cleveland Cavaliers,28300008,1983-10-28,CLE vs. NYK,0,240,106,...,0.657,16.0,19.0,35,26,9.0,3,17,33,NaN
2,21983,1610612749,MIL,Milwaukee Bucks,28300001,1983-10-28,MIL vs. IND,1,240,104,...,0.619,24.0,29.0,53,27,7.0,4,19,22,NaN
3,21983,1610612758,KCK,Kansas City Kings,28300002,1983-10-28,KCK vs. LAL,0,240,107,...,0.760,9.0,29.0,38,22,6.0,5,17,16,NaN
4,21983,1610612760,SEA,Seattle SuperSonics,28300004,1983-10-28,SEA vs. GOS,0,240,109,...,0.703,21.0,26.0,47,26,11.0,2,19,34,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42404,22022,1610612758,SAC,Sacramento Kings,22200350,2022-12-04,SAC vs. CHI,1,240,110,...,0.842,7.0,32.0,39,17,9.0,5,16,16,-9.0
42405,22022,1610612757,POR,Portland Trail Blazers,22200351,2022-12-04,POR vs. IND,1,241,116,...,0.833,12.0,32.0,44,22,10.0,5,15,22,-16.2
42406,22022,1610612752,NYK,New York Knicks,22200348,2022-12-04,NYK vs. CLE,1,239,92,...,0.750,8.0,31.0,39,17,7.0,6,17,15,-11.0
42407,22022,1610612765,DET,Detroit Pistons,22200347,2022-12-04,DET vs. MEM,0,240,112,...,0.724,12.0,34.0,46,28,9.0,5,13,21,10.0


In [62]:
#Calculate Cum AVGs
from timeit import default_timer as timer
from datetime import timedelta
import time

start = timer()

newColumns = ['GAME_NUMBER', 'TEAM_NAME', 'GAME_ID', 'SEASON_ID', 'GAME_DATE', 'LAST_GAME_DAYS', 'HOME_STREAK', 'AWAY_STREAK', 'WL', 'WL_AVG', 'PTS_AVG', 'FGM_AVG', 'FGA_AVG', 'FG3M_AVG', 'FG3A_AVG', 'FTM_AVG',
            'FTA_AVG', 'OREB_AVG', 'DREB_AVG', 'REB_AVG', 'AST_AVG', 'STL_AVG', 'BLK_AVG', 
              'TOV_AVG', 'PF_AVG']

seasons = gamesByTeams['SEASON_ID'].unique()
means = []
theGame = []
for i, season in enumerate(seasons):
    scores = gamesByTeams[gamesByTeams['SEASON_ID'] == season]
    scores = scores.sort_values(by=['GAME_DATE'], ascending=False)
    teams  = scores['TEAM_NAME'].unique()
    print(f"------{str(int(season) % 20000)}------")
    for team in teams:
        homeStreak = 0
        awayStreak = 0
        lastGameDate = None
        print(team)
        teamGames = scores[(scores['TEAM_NAME'] == team)]
        teamGames = teamGames.sort_values(by=['GAME_DATE'], ascending=False)
        teamGames = teamGames.reindex(index=teamGames.index[::-1]).reset_index(drop=True)
        newDf = pd.DataFrame({}, columns=newColumns)
        for j, game in teamGames.iterrows():
            newList = []
            if 'vs' in game['MATCHUP']:
                homeStreak += 1
                awayStreak = 0
            else:
                homeStreak = 0
                awayStreak += 1
            if j == 0:
                days = 0
            else:
                d1 = datetime.strptime(lastGameDate, '%Y-%m-%d')
                d2 = datetime.strptime(game['GAME_DATE'], '%Y-%m-%d')
                delta = d2 - d1
                days = delta.days 
            lastGameDate = game['GAME_DATE']
            teamAvg = teamGames.iloc[0:j][['WL', 'PTS', 'FGM', 'FGA', 'FG3M', 'FG3A',
            'FTM', 'FTA', 'OREB', 'DREB','REB', 'AST', 'STL', 'BLK', 'TOV',
            'PF']].mean()
            newList.append(j+1)
            newList.append(game['TEAM_NAME'])
            newList.append(game['GAME_ID'])
            newList.append(game['SEASON_ID'])
            newList.append(game['GAME_DATE'])
            newList.append(days)
            newList.append(homeStreak)
            newList.append(awayStreak)
            newList.append(game['WL'])
            newList.append(teamAvg['WL'] if j > 0 else 0)
            newList.append(teamAvg['PTS'] if j > 0 else 0)
            newList.append(teamAvg['FGM'] if j > 0 else 0)
            newList.append(teamAvg['FGA'] if j > 0 else 0)
            newList.append(teamAvg['FG3M'] if j > 0 else 0)
            newList.append(teamAvg['FG3A'] if j > 0 else 0)
            newList.append(teamAvg['FTM'] if j > 0 else 0)
            newList.append(teamAvg['FTA'] if j > 0 else 0)
            newList.append(teamAvg['OREB'] if j > 0 else 0)
            newList.append(teamAvg['DREB'] if j > 0 else 0)
            newList.append(teamAvg['REB'] if j > 0 else 0)
            newList.append(teamAvg['AST'] if j > 0 else 0)
            newList.append(teamAvg['STL'] if j > 0 else 0)
            newList.append(teamAvg['BLK'] if j > 0 else 0)
            newList.append(teamAvg['TOV'] if j > 0 else 0)
            newList.append(teamAvg['PF'] if j > 0 else 0)
            row   = pd.DataFrame([newList], columns=newColumns)
            newDf = pd.concat([newDf, row])
        newDf.to_csv(f"nba/computed_avgs/{team.replace(' ', '_').replace('/', '')}_cum_game_avg_{str(int(season) % 20000)}.csv") 
end = timer()
print(timedelta(seconds=end-start)) 

------2021------
Memphis Grizzlies
Los Angeles Lakers
Indiana Pacers
Detroit Pistons
Washington Wizards
Golden State Warriors
Orlando Magic
San Antonio Spurs
LA Clippers
Brooklyn Nets
Utah Jazz
Minnesota Timberwolves
Houston Rockets
New York Knicks
Atlanta Hawks
Chicago Bulls
Philadelphia 76ers
Oklahoma City Thunder
Sacramento Kings
Portland Trail Blazers
Cleveland Cavaliers
Charlotte Hornets
Boston Celtics
Milwaukee Bucks
Miami Heat
Phoenix Suns
Dallas Mavericks
Denver Nuggets
New Orleans Pelicans
Toronto Raptors
------2022------
Los Angeles Lakers
Portland Trail Blazers
Denver Nuggets
Brooklyn Nets
Phoenix Suns
San Antonio Spurs
Sacramento Kings
Cleveland Cavaliers
Indiana Pacers
Boston Celtics
Chicago Bulls
New Orleans Pelicans
Detroit Pistons
Washington Wizards
New York Knicks
Memphis Grizzlies
Oklahoma City Thunder
Minnesota Timberwolves
Orlando Magic
Toronto Raptors
LA Clippers
Milwaukee Bucks
Golden State Warriors
Houston Rockets
Charlotte Hornets
Utah Jazz
Dallas Mavericks
Atla

In [42]:
# Get Attendance

from timeit import default_timer as timer
from datetime import timedelta
import time

start = timer()
nba_months = ['october','november','december','january','february','march','april']
tables = pd.read_html(f"https://www.basketball-reference.com/leagues/NBA_2022_games-october.html")
columns = tables[0].columns
attendances = pd.DataFrame({}, columns=columns)
for year in range(1883, 2024):
    for month in nba_months:
        try:
            url = f"https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html"
            print(url);
            tables = pd.read_html(f"https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html")
            attendances = pd.concat([attendances, tables[0]])
            time.sleep(12)
        except:
            time.sleep(12)
            continue
attendances['Date'] = [datetime.strptime(game['Date'], '%a, %b %d, %Y') for index, game in attendances.iterrows()]
attendances.to_csv('nba-attendances-current-season.csv')

end = timer()
print(timedelta(seconds=end-start)) 

https://www.basketball-reference.com/leagues/NBA_2023_games-october.html
https://www.basketball-reference.com/leagues/NBA_2023_games-november.html
https://www.basketball-reference.com/leagues/NBA_2023_games-december.html
https://www.basketball-reference.com/leagues/NBA_2023_games-january.html
https://www.basketball-reference.com/leagues/NBA_2023_games-february.html
https://www.basketball-reference.com/leagues/NBA_2023_games-march.html
https://www.basketball-reference.com/leagues/NBA_2023_games-april.html
0:01:26.442576


In [56]:
attendances

,Date,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 5,Unnamed: 6,Attend.,Arena,Notes,Start (ET),Unnamed: 7
0,"Fri, Oct 28, 1983",Utah Jazz,125,Denver Nuggets,139,Box Score,NaN,10439,NaN,NaN,NaN,NaN
1,"Fri, Oct 28, 1983",Boston Celtics,121,Detroit Pistons,127,Box Score,NaN,17117,NaN,NaN,NaN,NaN
2,"Fri, Oct 28, 1983",Los Angeles Lakers,117,Kansas City Kings,107,Box Score,NaN,15782,NaN,NaN,NaN,NaN
3,"Fri, Oct 28, 1983",Indiana Pacers,83,Milwaukee Bucks,104,Box Score,NaN,10321,NaN,NaN,NaN,NaN
4,"Fri, Oct 28, 1983",Atlanta Hawks,108,New Jersey Nets,126,Box Score,NaN,10173,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1,"Sun, Jun 5, 2022",Boston Celtics,88,Golden State Warriors,107,NaN,Box Score,18064,Chase Center,NaN,8:00p,NaN
2,"Wed, Jun 8, 2022",Golden State Warriors,100,Boston Celtics,116,NaN,Box Score,19156,TD Garden,NaN,9:00p,NaN
3,"Fri, Jun 10, 2022",Golden State Warriors,107,Boston Celtics,97,NaN,Box Score,19156,TD Garden,NaN,9:00p,NaN
4,"Mon, Jun 13, 2022",Boston Celtics,94,Golden State Warriors,104,NaN,Box Score,18064,Chase Center,NaN,9:00p,NaN


In [ ]:
# Add Attendance to boxscore
from timeit import default_timer as timer
from datetime import timedelta
import time

start = timer()
games_attendance = []

for index, score in transformed_games.iterrows():
    game = attendances.loc[(attendances['Home/Neutral'] ==  score['HOME_TEAM_NAME']) 
                          & (attendances['Visitor/Neutral'] ==  score['AWAY_TEAM_NAME']) 
                          & (attendances['Date'] == score['HOME_GAME_DATE'])]
    if len(game) == 1:
        games_attendance.append(game.iloc[0, 8])
    else:
        games_attendance.append(np.nan)
transformed_games['ATTENDANCE'] = games_attendance        
end = timer()
print(timedelta(seconds=end-start)) 
transformed_games.to_csv('boxscores-now.csv')

In [5]:
pd.read_csv('boxscores-now.csv', index_col=0)

,HOME_SEASON_ID,HOME_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NAME,HOME_GAME_ID,HOME_GAME_DATE,HOME_MATCHUP,HOME_WL,HOME_MIN,HOME_PTS,...,AWAY_OREB,AWAY_DREB,AWAY_REB,AWAY_AST,AWAY_STL,AWAY_BLK,AWAY_TOV,AWAY_PF,AWAY_PLUS_MINUS,ATTENDANCE
0,22021,1610612749,MIL,Milwaukee Bucks,22100001,2021-10-19,MIL vs. BKN,W,239,127,...,5,39,44,19,3,9,12,17,-23.0,17341.0
1,22021,1610612747,LAL,Los Angeles Lakers,22100002,2021-10-19,LAL vs. GSW,L,241,114,...,9,41,50,30,9,2,17,18,7.0,18997.0
2,22021,1610612759,SAS,San Antonio Spurs,22100010,2021-10-20,SAS vs. ORL,W,239,123,...,13,31,44,17,6,6,16,20,-26.0,16697.0
3,22021,1610612762,UTA,Utah Jazz,22100011,2021-10-20,UTA vs. OKC,W,240,107,...,15,35,50,19,4,2,14,15,-21.0,18306.0
4,22021,1610612763,MEM,Memphis Grizzlies,22100007,2021-10-20,MEM vs. CLE,W,242,132,...,7,29,36,38,6,5,10,17,-11.0,15975.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1576,22022,1610612758,SAC,Sacramento Kings,22200350,2022-12-04,SAC vs. CHI,W,240,110,...,7,32,39,17,9,5,16,16,-9.0,17611.0
1577,22022,1610612757,POR,Portland Trail Blazers,22200351,2022-12-04,POR vs. IND,W,241,116,...,12,32,44,22,10,5,15,22,-16.2,17479.0
1578,22022,1610612752,NYK,New York Knicks,22200348,2022-12-04,NYK vs. CLE,W,239,92,...,8,31,39,17,7,6,17,15,-11.0,19007.0
1579,22022,1610612765,DET,Detroit Pistons,22200347,2022-12-04,DET vs. MEM,L,240,112,...,12,34,46,28,9,5,13,21,10.0,20088.0


In [80]:
#Create Cum Avgs for predictions
from datetime import timedelta
attendance['Date'] = [datetime.strptime(game['Date'], '%Y-%m-%d') for index, game in attendance.iterrows()]
teams = attendance['Home/Neutral'].unique()
newColumns = ['GAME_NUMBER', 'TEAM_NAME', 'GAME_ID', 'SEASON_ID', 'GAME_DATE', 'LAST_GAME_DAYS', 'HOME_STREAK', 'AWAY_STREAK', 'WL_AVG', 'PTS_AVG', 'FGM_AVG', 'FGA_AVG', 'FG3M_AVG', 'FG3A_AVG', 'FTM_AVG',
            'FTA_AVG', 'OREB_AVG', 'DREB_AVG', 'REB_AVG', 'AST_AVG', 'STL_AVG', 'BLK_AVG', 
              'TOV_AVG', 'PF_AVG']
for team in teams:
    teamName = 'LA Clippers' if team == 'Los Angeles Clippers' else team 
    yesterday = datetime.today() - timedelta(days = 2)
    teamCumAvgs = pd.read_csv(f"nba/computed_avgs/{teamName.replace(' ', '_').replace('/', '')}_cum_game_avg_2022.csv").iloc[-1]
    schedule = attendance[((attendance['Home/Neutral'] == team) | (attendance['Visitor/Neutral'] == team)) & (attendance['Date'] > yesterday.strftime('%Y-%m-%d'))]
    homeStreak = teamCumAvgs['HOME_STREAK']
    awayStreak = teamCumAvgs['AWAY_STREAK']
    lastGameDate = datetime.strptime(teamCumAvgs['GAME_DATE'], '%Y-%m-%d')
    newDf = pd.DataFrame({}, columns=newColumns)
    for i, game in schedule.iterrows():
        newList = []
        if team == game['Home/Neutral']:
                homeStreak += 1
                awayStreak = 0
        else:
            homeStreak = 0
            awayStreak += 1
        delta = datetime.strptime(game['Date'], '%Y-%m-%d') - lastGameDate
        days = delta.days 
        newList.append(teamCumAvgs['GAME_NUMBER']+(i+1))
        newList.append(team)
        newList.append(teamCumAvgs['GAME_ID'])
        newList.append(teamCumAvgs['SEASON_ID'])
        newList.append(game['Date'])
        newList.append(days)
        newList.append(homeStreak)
        newList.append(awayStreak)
        newList.append(teamCumAvgs['WL_AVG'])
        newList.append(teamCumAvgs['PTS_AVG'])
        newList.append(teamCumAvgs['FGM_AVG'])
        newList.append(teamCumAvgs['FGA_AVG'])
        newList.append(teamCumAvgs['FG3M_AVG'])
        newList.append(teamCumAvgs['FG3A_AVG'])
        newList.append(teamCumAvgs['FTM_AVG'])
        newList.append(teamCumAvgs['FTA_AVG'])
        newList.append(teamCumAvgs['OREB_AVG'])
        newList.append(teamCumAvgs['DREB_AVG'])
        newList.append(teamCumAvgs['REB_AVG'])
        newList.append(teamCumAvgs['AST_AVG'])
        newList.append(teamCumAvgs['STL_AVG'])
        newList.append(teamCumAvgs['BLK_AVG'])
        newList.append(teamCumAvgs['TOV_AVG'])
        newList.append(teamCumAvgs['PF_AVG'])
        row   = pd.DataFrame([newList], columns=newColumns)
        newDf = pd.concat([newDf, row])
        newDf.to_csv(f"nba/computed_avgs_preds/{team.replace(' ', '_').replace('/', '')}_cum_game_avg_2022.csv")
        lastGameDate = datetime.strptime(game['Date'], '%Y-%m-%d')

In [91]:
# Create prediction DF

newColumns = ['HOME_TEAM_NAME', 'AWAY_TEAM_NAME','GAME_ID','SEASON_ID' ,'DATE','HOME_TEAM_GAME_NUMBER', 'HOME_LAST_GAME_DAYS', 'HOME_HOME_STREAK', 'HOME_AWAY_STREAK', 'HOME_WL_AVG', 'HOME_PTS_AVG', 'HOME_FGM_AVG', 'HOME_FGA_AVG', 'HOME_FG3M_AVG', 'HOME_FG3A_AVG', 'HOME_FTM_AVG',
            'HOME_FTA_AVG', 'HOME_OREB_AVG', 'HOME_DREB_AVG', 'HOME_REB_AVG', 'HOME_AST_AVG', 'HOME_STL_AVG', 'HOME_BLK_AVG', 'HOME_TOV_AVG', 'HOME_PF_AVG',
            'AWAY_TEAM_GAME_NUMBER', 'AWAY_LAST_GAME_DAYS', 'AWAY_HOME_STREAK', 'AWAY_AWAY_STREAK','AWAY_WL_AVG', 'AWAY_PTS_AVG', 'AWAY_FGM_AVG', 'AWAY_FGA_AVG', 'AWAY_FG3M_AVG', 'AWAY_FG3A_AVG',
            'AWAY_FTM_AVG', 'AWAY_FTA_AVG', 'AWAY_OREB_AVG', 'AWAY_DREB_AVG','AWAY_REB_AVG', 'AWAY_AST_AVG', 'AWAY_STL_AVG', 'AWAY_BLK_AVG', 'AWAY_TOV_AVG',
            'AWAY_PF_AVG']
newDf = pd.DataFrame({}, columns=newColumns)
yesterday = datetime.today() - timedelta(days = 2)
schedule = attendance[attendance['Date'] > yesterday.strftime('%Y-%m-%d')]
for i, game in schedule.iterrows():
    newList  = []
    homeTeam = pd.read_csv(f"nba/computed_avgs_preds/{game['Home/Neutral'].replace(' ', '_').replace('/', '')}_cum_game_avg_2022.csv")
    awayTeam = pd.read_csv(f"nba/computed_avgs_preds/{game['Visitor/Neutral'].replace(' ', '_').replace('/', '')}_cum_game_avg_2022.csv")
    homeTeam = homeTeam.loc[homeTeam['GAME_DATE'] == game['Date']].iloc[0]
    awayTeam = awayTeam.loc[awayTeam['GAME_DATE'] == game['Date']].iloc[0]
    newList  = []
    newList.append(homeTeam['TEAM_NAME'])
    newList.append(awayTeam['TEAM_NAME'])
    newList.append(homeTeam['GAME_ID'])
    newList.append(homeTeam['SEASON_ID'])
    newList.append(game['Date'])
    newList.append(homeTeam['GAME_NUMBER'])
    newList.append(homeTeam['LAST_GAME_DAYS'])
    newList.append(homeTeam['HOME_STREAK'])
    newList.append(homeTeam['AWAY_STREAK'])
    newList.append(homeTeam['WL_AVG'])
    newList.append(homeTeam['PTS_AVG'])
    newList.append(homeTeam['FGM_AVG'])
    newList.append(homeTeam['FGA_AVG'])
    newList.append(homeTeam['FG3M_AVG'])
    newList.append(homeTeam['FG3A_AVG'])
    newList.append(homeTeam['FTM_AVG'])
    newList.append(homeTeam['FTA_AVG'])
    newList.append(homeTeam['OREB_AVG'])
    newList.append(homeTeam['DREB_AVG'])
    newList.append(homeTeam['REB_AVG'])
    newList.append(homeTeam['AST_AVG'])
    newList.append(homeTeam['STL_AVG'])
    newList.append(homeTeam['BLK_AVG'])
    newList.append(homeTeam['TOV_AVG'])
    newList.append(homeTeam['PF_AVG'])
    newList.append(awayTeam['GAME_NUMBER'])
    newList.append(awayTeam['LAST_GAME_DAYS'])
    newList.append(awayTeam['HOME_STREAK'])
    newList.append(awayTeam['AWAY_STREAK'])
    newList.append(awayTeam['WL_AVG'])
    newList.append(awayTeam['PTS_AVG'])
    newList.append(awayTeam['FGM_AVG'])
    newList.append(awayTeam['FGA_AVG'])
    newList.append(awayTeam['FG3M_AVG'])
    newList.append(awayTeam['FG3A_AVG'])
    newList.append(awayTeam['FTM_AVG'])
    newList.append(awayTeam['FTA_AVG'])
    newList.append(awayTeam['OREB_AVG'])
    newList.append(awayTeam['DREB_AVG'])
    newList.append(awayTeam['REB_AVG'])
    newList.append(awayTeam['AST_AVG'])
    newList.append(awayTeam['STL_AVG'])
    newList.append(awayTeam['BLK_AVG'])
    newList.append(awayTeam['TOV_AVG'])
    newList.append(awayTeam['PF_AVG'])
    row   = pd.DataFrame([newList], columns=newColumns)
    newDf = pd.concat([newDf, row])
newDf.reset_index(drop=True).to_csv(f"league_cum_game_avgs_preds.csv") 

In [92]:
newDf

,HOME_TEAM_NAME,AWAY_TEAM_NAME,GAME_ID,SEASON_ID,DATE,HOME_TEAM_GAME_NUMBER,HOME_LAST_GAME_DAYS,HOME_HOME_STREAK,HOME_AWAY_STREAK,HOME_WL_AVG,...,AWAY_FTM_AVG,AWAY_FTA_AVG,AWAY_OREB_AVG,AWAY_DREB_AVG,AWAY_REB_AVG,AWAY_AST_AVG,AWAY_STL_AVG,AWAY_BLK_AVG,AWAY_TOV_AVG,AWAY_PF_AVG
0,Charlotte Hornets,Los Angeles Clippers,22200339,22022,2022-12-05,51,2,3,0,0.318182,...,17.478261,23.565217,8.565217,34.478261,43.043478,22.695652,6.652174,4.782609,15.217391,20.217391
0,Orlando Magic,Milwaukee Bucks,22200340,22022,2022-12-05,53,2,1,0,0.217391,...,17.142857,23.761905,11.761905,36.666667,48.428571,24.666667,6.714286,6.333333,13.714286,19.238095
0,Atlanta Hawks,Oklahoma City Thunder,22200327,22022,2022-12-05,53,3,2,0,0.545455,...,16.909091,21.590909,12.136364,32.590909,44.727273,24.090909,8.045455,5.954545,13.318182,22.045455
0,Toronto Raptors,Boston Celtics,22200340,22022,2022-12-05,54,2,2,0,0.500000,...,18.826087,22.217391,8.000000,33.913043,41.913043,27.434783,5.869565,5.000000,13.217391,20.521739
0,Houston Rockets,Philadelphia 76ers,22200342,22022,2022-12-05,55,2,1,0,0.272727,...,18.590909,23.000000,8.136364,31.727273,39.863636,23.318182,8.454545,4.636364,13.772727,19.409091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Los Angeles Lakers,Utah Jazz,22200349,22022,2023-04-09,88,2,2,0,0.428571,...,17.400000,22.360000,11.920000,32.200000,44.120000,27.360000,7.320000,4.920000,15.520000,21.560000
0,Minnesota Timberwolves,New Orleans Pelicans,22200341,22022,2023-04-09,90,1,1,0,0.500000,...,18.818182,23.681818,11.500000,34.045455,45.545455,27.045455,8.772727,4.454545,14.227273,19.500000
0,Oklahoma City Thunder,Memphis Grizzlies,22200341,22022,2023-04-09,91,3,1,0,0.409091,...,17.545455,25.136364,14.272727,34.818182,49.090909,24.136364,6.863636,5.363636,14.045455,20.000000
0,Phoenix Suns,Los Angeles Clippers,22200345,22022,2023-04-09,92,2,1,0,0.681818,...,17.478261,23.565217,8.565217,34.478261,43.043478,22.695652,6.652174,4.782609,15.217391,20.217391
